# Word2Vec - Treinamento de word embbeding

### Bibliotecas básicas e outras inicializações

In [1]:
import pandas as pd
import numpy as np
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
nlp = spacy.load('pt_core_news_sm')

### Carregando os dados

In [3]:
dados_treino = pd.read_csv('./datasets/treino.csv')
print(dados_treino.shape)
dados_treino.sample()

(90000, 6)


,title,text,date,category,subcategory,link
41226,Problema do Palmeiras é não ter projeto para c...,A folha de pagamento do Palmeiras está perto d...,2017-06-18,colunas,pvc,http://www1.folha.uol.com.br/colunas/pvc/2017/...


In [4]:
dados_teste = pd.read_csv('./datasets/teste.csv')
print(dados_teste.shape)
dados_teste.sample()

(20513, 6)


,title,text,date,category,subcategory,link
3084,Combinar com o mar,"RIO DE JANEIRO - Em janeiro, na véspera da ina...",2016-04-23,colunas,ruycastro,http://www1.folha.uol.com.br/colunas/ruycastro...


### Pré-processamento do dados

In [5]:
def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)
    
    if len(tokens_validos) > 2:
        return ' '.join(tokens_validos)

In [6]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino['title'])
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento, batch_size=1000, n_process=-1)]
titulos_tratados = pd.DataFrame({'titulo': textos_tratados})
titulos_tratados = titulos_tratados.dropna().drop_duplicates()
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron e le pen turno frança revés siglas trad...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço e alckmin anuncia que...
4,queda maio a atividade econômica sobe junho bc


### Treinamento Word2Vec

#### CBOW

In [7]:
from gensim.models import Word2Vec
w2v_modelo = Word2Vec(sg=0, window=2, size=300, min_count=5, alpha=0.03, min_alpha=0.007)
tokens = [titulo.split(' ') for titulo in titulos_tratados['titulo']]
w2v_modelo.build_vocab(tokens)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec

class callback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss após a época {}: {}'.format(self.epoch, loss))
        else:
            print('Loss após a época {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss


w2v_modelo.train(tokens, total_examples=w2v_modelo.corpus_count, epochs=30, compute_loss=True, callbacks=[callback()]) 

Loss após a época 0: 629875.875
Loss após a época 1: 503941.5
Loss após a época 2: 409089.5
Loss após a época 3: 375963.5
Loss após a época 4: 362940.625
Loss após a época 5: 318465.75
Loss após a época 6: 301026.25
Loss após a época 7: 299670.0
Loss após a época 8: 295305.5
Loss após a época 9: 254407.5
Loss após a época 10: 255349.75
Loss após a época 11: 237791.75
Loss após a época 12: 210967.5
Loss após a época 13: 202848.5
Loss após a época 14: 201810.5
Loss após a época 15: 194977.0
Loss após a época 16: 180257.0
Loss após a época 17: 173183.0
Loss após a época 18: 158873.0
Loss após a época 19: 162802.5
Loss após a época 20: 148605.5
Loss após a época 21: 152450.0
Loss após a época 22: 141840.0
Loss após a época 23: 144707.5
Loss após a época 24: 148800.0
Loss após a época 25: 139279.0
Loss após a época 26: 128141.0
Loss após a época 27: 131237.5
Loss após a época 28: 128627.5
Loss após a época 29: 128205.0


(14993931, 17644710)

In [9]:
w2v_modelo.wv.most_similar('google')

[('facebook', 0.6001596450805664),
 ('apple', 0.5977469682693481),
 ('software', 0.5043631792068481),
 ('uber', 0.47655439376831055),
 ('airbnb', 0.4682568311691284),
 ('amazon', 0.46755990386009216),
 ('apps', 0.462656706571579),
 ('yahoo', 0.45818811655044556),
 ('waze', 0.4439406991004944),
 ('fbi', 0.4402138590812683)]

#### SKIP-GRAM

In [10]:
w2v_modelo_sg = Word2Vec(sg=1, window=5, size=300, min_count=5, alpha=0.03, min_alpha=0.007)
w2v_modelo_sg.build_vocab(tokens)
w2v_modelo_sg.train(tokens, total_examples=w2v_modelo.corpus_count, epochs=30, compute_loss=True, callbacks=[callback()]) 

Loss após a época 0: 1928031.125
Loss após a época 1: 1516673.375
Loss após a época 2: 1318825.5
Loss após a época 3: 1155002.5
Loss após a época 4: 1119180.5
Loss após a época 5: 1038607.5
Loss após a época 6: 962070.5
Loss após a época 7: 900627.0
Loss após a época 8: 878897.0
Loss após a época 9: 858729.0
Loss após a época 10: 880373.0
Loss após a época 11: 855779.0
Loss após a época 12: 836868.0
Loss após a época 13: 779539.0
Loss após a época 14: 799678.0
Loss após a época 15: 750945.0
Loss após a época 16: 613878.0
Loss após a época 17: 556666.0
Loss após a época 18: 545234.0
Loss após a época 19: 529814.0
Loss após a época 20: 516806.0
Loss após a época 21: 511364.0
Loss após a época 22: 501940.0
Loss após a época 23: 489488.0
Loss após a época 24: 482034.0
Loss após a época 25: 474102.0
Loss após a época 26: 468218.0
Loss após a época 27: 459906.0
Loss após a época 28: 449560.0
Loss após a época 29: 449476.0


(14993330, 17644710)

In [11]:
w2v_modelo_sg.wv.most_similar('google')

[('apple', 0.42483484745025635),
 ('buffett', 0.4066280126571655),
 ('android', 0.4054515063762665),
 ('reguladores', 0.4016849100589752),
 ('waze', 0.39828798174858093),
 ('patentes', 0.39816367626190186),
 ('facebook', 0.3842591643333435),
 ('app', 0.3688926100730896),
 ('anunciantes', 0.36745041608810425),
 ('bmw', 0.36109450459480286)]

#### Salva os modelos

In [12]:
w2v_modelo.wv.save_word2vec_format('models/modelo_cbow.txt', binary=False)
w2v_modelo_sg.wv.save_word2vec_format('models/modelo_skipgram.txt', binary=False)

### Classificador

#### Carrega os modelos

In [33]:
from gensim.models import KeyedVectors
w2v_modelo = KeyedVectors.load_word2vec_format('models/modelo_cbow.txt')
w2v_modelo_sg = KeyedVectors.load_word2vec_format('models/modelo_skipgram.txt')

#### Vetorizando os dados

In [34]:
nlp = spacy.load('pt_core_news_sm', disable=['paser', 'ner', 'tagger', 'textcat'])

def tokenizador(texto):    
    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text.lower()) 
    return  tokens_validos


def combinacao_de_vetores_por_soma(palavras, modelo):
    vetor_resultante = np.zeros((1, 300))
    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)
        except KeyError:
            pass
    return vetor_resultante



def matriz_vetores(textos, modelo):
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))
    for i in range(x):
        palavras = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)
    return matriz

In [37]:
matriz_vetores_treino_cbow = matriz_vetores(dados_treino['title'], w2v_modelo)
print(matriz_vetores_treino_cbow.shape)
matriz_vetores_teste_cbow = matriz_vetores(dados_teste['title'], w2v_modelo)
print(matriz_vetores_teste_cbow.shape)

(90000, 300)
(20513, 300)


#### Classificando os dados

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(modelo, x_treino, y_treino, x_teste, y_teste):
    model = LogisticRegression(max_iter=800)
    model.fit(x_treino, y_treino)
    categorias = model.predict(x_teste)
    resultados = classification_report(y_teste, categorias)
    print(resultados)
    return model

CBOW

In [40]:
RL_cbow = classificador(w2v_modelo, 
                        matriz_vetores_treino_cbow, 
                        dados_treino['category'], 
                        matriz_vetores_teste_cbow, dados_teste['category'])

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.63      0.80      0.71      1698
     esporte       0.93      0.87      0.90      4663
   ilustrada       0.13      0.83      0.22       131
     mercado       0.83      0.78      0.81      5867
       mundo       0.74      0.84      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.80      0.70     20513
weighted avg       0.82      0.79      0.80     20513



SKIP-GRAM

In [41]:
matriz_vetores_treino_sg = matriz_vetores(dados_treino['title'], w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(dados_teste['title'], w2v_modelo_sg)

RL_skip = classificador(w2v_modelo_sg, 
                        matriz_vetores_treino_sg, 
                        dados_treino['category'], 
                        matriz_vetores_teste_sg, dados_teste['category'])

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.93      0.87      0.90      4663
   ilustrada       0.14      0.86      0.24       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.75      0.84      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.69      0.81      0.70     20513
weighted avg       0.82      0.79      0.80     20513



Salvando os modelos

In [43]:
import pickle
with open('models/rl_cbow.pkl', 'wb') as f:
    pickle.dump(RL_cbow, f)
    
with open('models/rl_sg.pkl', 'wb') as f:
    pickle.dump(RL_skip, f)